In [206]:
import numpy as np
import pandas as pd
import datetime

sessionData = pd.DataFrame()
lastid = -1
lastClickData = pd.DataFrame()

sChunk = 10000 # default chunk size of searching.
def dateparse(dt_str):
    return datetime.datetime.strptime(dt_str, '%Y-%m-%dT%H:%M:%S.%fZ')

def getSessionData(sid,part_size = sChunk):
    # search for specific session ids.
    #part_size = 10000
    #sid = 16004 # session id to search.
    #print("Search partition size: "+str(part_size),end="\r")
    sres = pd.DataFrame()
    s = pd.read_csv("yoochoose-data\yoochoose-clicks.dat",parse_dates = [1], date_parser = dateparse, chunksize=part_size)
    index = 0
    for chunk in s:
        sres=chunk.loc[chunk['session'] == sid]
        if not sres.empty:
            #print("\nFound session id in partition "+str(index))
            widedf = pd.read_csv("yoochoose-data\yoochoose-clicks.dat", skiprows=range(1,part_size*(index)-int(0.5*part_size)),nrows=part_size*2,
                             parse_dates = [1], date_parser = dateparse)
            sres=widedf.loc[widedf['session'] == sid]
            break
        else:
            loadSpin=['|','/','-','\\']
            print(loadSpin[int(index/2)%4]+" Searching partition "+str(index)+"...",end="\r")
        index+=1
    return sres

def parseSession(session,time,item_id):
    print("======== Session "+str(session)+" ========")
    print("# Product:",item_id)
    print("# Purchase time:",time)
    global lastid, lastClickData
    counter = 0
    clickTime = []
    tgap = datetime.datetime.now()
    if session != lastid:
        lastid = session
        temp = getSessionData(session,10000)
        lastClickData=temp.loc[temp['item_id'] == item_id]
    for index, srow in lastClickData.iterrows():
        if srow['timestamp'] < time:
            counter+=1
        clickTime.append(srow['timestamp'])
    print("Clicks before buy: "+str(counter)+"  |  First click: "+str(clickTime[0])+"  |  Last click: "+str(clickTime[-1]))
    print("First click until buy: "+str((time - clickTime[0]).total_seconds())+
          " sec.  |  Last click until buy: "+str((time - clickTime[-1]).total_seconds())+" sec.",end="\n\n")

# iterate through all buy sessions.
def getDataA(length=-1):
    loop = 0
    for index,row in buys.iterrows():
        if loop >= length and length != -1:
            break;
        parseSession(row['session'],row['timestamp'],row['item_id'])
        loop += 1
        
# get data by item_id
def getDataI(itemSID,length=-1):
    loop = 0
    df = buys.loc[buys['item_id'] == itemSID]
    if df.empty:
        print("Find by Item ID: No Result!")
    else:
        for index,row in df.iterrows():
            if loop >= length and length != -1:
                break;
            parseSession(row['session'],row['timestamp'],row['item_id'])
            loop += 1

# get data by session ID
def getDataS(sessionSID,length=-1):
    loop = 0
    df = buys.loc[buys['session'] == sessionSID]
    if df.empty:
        print("Find by Session: No Result!")
    else:
        for index,row in df.iterrows():
            if loop >= length and length != -1:
                break;
            parseSession(row['session'],row['timestamp'],row['item_id'])
            loop += 1

In [178]:
# Read and sort buy session data
buys = pd.read_csv("yoochoose-data\yoochoose-buys.dat",parse_dates = [1], date_parser = dateparse)
buys=buys.sort_values(by='session')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1150753 entries, 0 to 1150752
Data columns (total 5 columns):
 #   Column     Non-Null Count    Dtype         
---  ------     --------------    -----         
 0   session    1150753 non-null  int64         
 1   timestamp  1150753 non-null  datetime64[ns]
 2   item_id    1150753 non-null  int64         
 3   price      1150753 non-null  int64         
 4   quantity   1150753 non-null  int64         
dtypes: datetime64[ns](1), int64(4)
memory usage: 43.9 MB


In [207]:
getDataA(3)
getDataI(214706441,3)
getDataS(420374,3) 

======== Session 11 ========
# Product: 214821371
# Purchase time: 2014-04-03 11:04:18.097000
Clicks before buy: 3  |  First click: 2014-04-03 10:45:29.873000  |  Last click: 2014-04-03 10:46:57.355000
First click until buy: 1128.224 sec.  |  Last click until buy: 1040.742 sec.

======== Session 11 ========
# Product: 214821371
# Purchase time: 2014-04-03 11:04:11.417000
Clicks before buy: 3  |  First click: 2014-04-03 10:45:29.873000  |  Last click: 2014-04-03 10:46:57.355000
First click until buy: 1121.544 sec.  |  Last click until buy: 1034.062 sec.

======== Session 12 ========
# Product: 214717867
# Purchase time: 2014-04-02 10:42:17.227000
Clicks before buy: 2  |  First click: 2014-04-02 10:30:13.176000  |  Last click: 2014-04-02 10:33:12.621000
First click until buy: 724.051 sec.  |  Last click until buy: 544.606 sec.

======== Session 33 ========
# Product: 214706441
# Purchase time: 2014-04-06 18:24:51.426000
Clicks before buy: 4  |  First click: 2014-04-06 18:17:33.199000  | 